<a href="https://colab.research.google.com/github/Osondu-ifunanya/Watershed-prioritization-using-ANN-based-multicriteria-decision-analysis/blob/main/Watershed_Prioritization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.inspection import permutation_importance

np.random.seed(42)

# -----------------------------
# 1) Synthetic data generation
# -----------------------------
n_ws = 150  # number of sub-watersheds
ws_id = [f"WS_{i:03d}" for i in range(1, n_ws+1)]

# Generate synthetic criteria (ranges chosen to look realistic)
slope_mean = np.random.uniform(2, 25, n_ws)                 # degrees (cost)
relief_ratio = np.random.uniform(0.02, 0.25, n_ws)          # (cost)
drainage_density = np.random.uniform(0.5, 3.5, n_ws)        # km/km^2 (cost)
stream_frequency = np.random.uniform(0.2, 4.0, n_ws)        # no./km^2 (cost)
rainfall = np.random.uniform(600, 1800, n_ws)               # mm/yr (cost for erosion/flood risk)
soil_erod_k = np.random.uniform(0.05, 0.45, n_ws)           # USLE K-factor (cost)
ndvi = np.random.uniform(0.1, 0.8, n_ws)                    # (benefit)
forest_pct = np.random.uniform(5, 85, n_ws)                 # % area (benefit)
builtup_pct = np.random.uniform(0, 40, n_ws)                # % area (cost)
curve_number = np.random.uniform(55, 92, n_ws)              # CN (cost)
runoff_coeff = np.random.uniform(0.05, 0.75, n_ws)          # (cost)
dist_stream_km = np.random.uniform(0.1, 6.0, n_ws)          # mean distance to streams (benefit)
lith_perm_index = np.random.uniform(0.1, 0.9, n_ws)         # lithological permeability (benefit)
gw_potential = np.random.uniform(0.1, 0.9, n_ws)            # groundwater potential (benefit)
pop_density = np.random.uniform(50, 2500, n_ws)             # persons/km^2 (cost)
erosion_risk = np.random.uniform(0.1, 1.0, n_ws)            # composite index (cost)

# Assemble into DataFrame
features = pd.DataFrame({
    "slope_mean_deg": slope_mean,
    "relief_ratio": relief_ratio,
    "drainage_density": drainage_density,
    "stream_frequency": stream_frequency,
    "rainfall_mm": rainfall,
    "soil_erod_k": soil_erod_k,
    "ndvi": ndvi,
    "forest_pct": forest_pct,
    "builtup_pct": builtup_pct,
    "curve_number": curve_number,
    "runoff_coeff": runoff_coeff,
    "dist_stream_km": dist_stream_km,
    "lith_perm_index": lith_perm_index,
    "gw_potential": gw_potential,
    "pop_density": pop_density,
    "erosion_risk": erosion_risk
}, index=ws_id)

# Directions: +1 = benefit (higher is better -> lowers priority), -1 = cost (higher is worse -> increases priority)
directions = {
    "slope_mean_deg": -1,
    "relief_ratio": -1,
    "drainage_density": -1,
    "stream_frequency": -1,
    "rainfall_mm": -1,          # assume higher rainfall worsens erosion/flood risk
    "soil_erod_k": -1,
    "ndvi": +1,
    "forest_pct": +1,
    "builtup_pct": -1,
    "curve_number": -1,
    "runoff_coeff": -1,
    "dist_stream_km": +1,       # farther from streams = less immediate risk
    "lith_perm_index": +1,
    "gw_potential": +1,
    "pop_density": -1,
    "erosion_risk": -1
}

# Create a "true" latent priority score from plausible relationships (higher score = higher priority)
# Weights sum not required; add noise to simulate label uncertainty.
true_w = {
    "slope_mean_deg": 0.12,
    "relief_ratio": 0.08,
    "drainage_density": 0.10,
    "stream_frequency": 0.07,
    "rainfall_mm": 0.06,
    "soil_erod_k": 0.12,
    "ndvi": -0.10,
    "forest_pct": -0.08,
    "builtup_pct": 0.06,
    "curve_number": 0.08,
    "runoff_coeff": 0.07,
    "dist_stream_km": -0.04,
    "lith_perm_index": -0.03,
    "gw_potential": -0.04,
    "pop_density": 0.05,
    "erosion_risk": 0.18
}

# Normalize each feature (z-score) to make the linear synthetic function stable
f_mu = features.mean(axis=0)
f_sd = features.std(axis=0).replace(0, 1.0)
z = (features - f_mu) / f_sd

# Latent true priority score (linear combo in z-space + noise)
latent = np.zeros(n_ws)
for k, w in true_w.items():
    latent += w * z[k].values
noise = np.random.normal(0, 0.25, n_ws)
priority_true = latent + noise

# Scale to 0..1 for readability
priority_min, priority_max = priority_true.min(), priority_true.max()
priority_true_01 = (priority_true - priority_min) / (priority_max - priority_min)

# Priority classes by terciles
q1, q2 = np.quantile(priority_true_01, [1/3, 2/3])
def class_from_score(s):
    if s < q1: return "Low"
    if s < q2: return "Medium"
    return "High"

priority_class = np.array([class_from_score(s) for s in priority_true_01])

# -----------------------------
# 2) Train ANN (MLPRegressor)
# -----------------------------
X = features.values
y = priority_true_01  # continuous target (0..1)

X_train, X_test, y_train, y_test, id_train, id_test = train_test_split(
    X, y, features.index.values, test_size=0.25, random_state=42
)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# MLP configuration (compact, stable)
mlp = MLPRegressor(hidden_layer_sizes=(64, 32),
                   activation='relu',
                   solver='adam',
                   learning_rate_init=1e-3,
                   alpha=1e-4,
                   max_iter=1500,
                   random_state=42)
mlp.fit(X_train_s, y_train)

# Evaluate
y_pred = mlp.predict(X_test_s)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"ANN test R2 = {r2:.3f}, RMSE = {rmse:.3f}")

# -----------------------------
# 3) ANN-driven MCDA weights via permutation importance
# -----------------------------
perm = permutation_importance(mlp, X_test_s, y_test, n_repeats=15, random_state=42)
imp = perm.importances_mean  # can be negative if noise; clip at 0
imp = np.clip(imp, 0, None)
# Normalize to sum to 1
weights_ann = imp / (imp.sum() + 1e-12)
weight_table = pd.DataFrame({
    "criterion": features.columns,
    "ann_weight": weights_ann
}).sort_values("ann_weight", ascending=False)
print("\nDerived MCDA weights from ANN permutation importance:")
print(weight_table)

# -----------------------------
# 4) Compute MCDA composite score (benefit/cost aware)
# -----------------------------
# Normalize all features with z-score then convert to positive direction using 'directions'
Z_all = (features - f_mu) / f_sd

# Convert to benefit space: if criterion is cost (-1), multiply by -1 so larger is always better (low priority)
benefit_Z = Z_all.copy()
for k, d in directions.items():
    if d == -1:
        benefit_Z[k] = -benefit_Z[k]

# Rescale benefit_Z to 0..1 for each criterion (min-max)
benefit_01 = (benefit_Z - benefit_Z.min()) / (benefit_Z.max() - benefit_Z.min() + 1e-12)

# Weighted sum using ANN-derived weights
w_vec = pd.Series(weights_ann, index=features.columns)
mcda_score = (benefit_01 * w_vec).sum(axis=1)

# Convert MCDA to "priority" direction (higher value -> higher priority)
# Our benefit_01 means larger is better (lower priority), so invert:
mcda_priority = 1.0 - (mcda_score - mcda_score.min()) / (mcda_score.max() - mcda_score.min() + 1e-12)

# Rank and classify
rank_ann = mlp.predict(scaler.transform(features.values))  # ANN priority (0..1)
rank_ann_series = pd.Series(rank_ann, index=features.index, name="ann_priority_01")

def class_by_tercile(arr):
    q1, q2 = np.quantile(arr, [1/3, 2/3])
    return pd.Series(np.where(arr<q1, "Low", np.where(arr<q2, "Medium", "High")),
                     index=features.index)

ann_class = class_by_tercile(rank_ann_series.values)
mcda_class = class_by_tercile(mcda_priority.values)

# -----------------------------
# 5) Assemble outputs
# -----------------------------
out = features.copy()
out["priority_true_01"] = priority_true_01
out["priority_true_class"] = priority_class
out["ann_priority_01"] = rank_ann_series
out["ann_priority_class"] = ann_class
out["mcda_priority_01"] = mcda_priority
out["mcda_priority_class"] = mcda_class

# Rankings (1 = highest priority)
out["ann_rank"] = (-out["ann_priority_01"]).rank(method="dense").astype(int)
out["mcda_rank"] = (-out["mcda_priority_01"]).rank(method="dense").astype(int)

print("\nTop 10 watersheds by ANN priority:")
print(out.sort_values("ann_priority_01", ascending=False).head(10)[
    ["ann_priority_01","ann_rank"]
])

print("\nTop 10 watersheds by MCDA priority:")
print(out.sort_values("mcda_priority_01", ascending=False).head(10)[
    ["mcda_priority_01","mcda_rank"]
])

# -----------------------------
# 6) Plots
# -----------------------------
plt.figure(figsize=(7,4))
plt.scatter(y_test, y_pred, alpha=0.6)
mn, mx = min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())
plt.plot([mn, mx], [mn, mx], 'r--', lw=1)
plt.xlabel("True priority (0..1)")
plt.ylabel("ANN predicted priority (0..1)")
plt.title(f"ANN Performance (R²={r2:.2f}, RMSE={rmse:.2f})")
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,5))
weight_table.plot(kind="barh", x="criterion", y="ann_weight", legend=False)
plt.gca().invert_yaxis()
plt.xlabel("ANN-derived weight")
plt.title("MCDA Weights from ANN Permutation Importance")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,5))
plt.scatter(out["ann_priority_01"], out["mcda_priority_01"], alpha=0.7)
plt.xlabel("ANN priority (0..1)")
plt.ylabel("MCDA priority (0..1)")
plt.title("Agreement: ANN vs ANN-weighted MCDA")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# -----------------------------
# 7) Export to Excel
# -----------------------------
with pd.ExcelWriter("watershed_prioritization_ann_mcda.xlsx", engine="openpyxl") as writer:
    out.to_excel(writer, sheet_name="prioritization", index=True)
    weight_table.to_excel(writer, sheet_name="ann_mcda_weights", index=False)

print("\nExported results to: watershed_prioritization_ann_mcda.xlsx")

